In [ ]:
import os
os.environ['http_proxy'] = '127.0.0.1:7890'
os.environ['https_proxy'] = '127.0.0.1:7890'

## basics

- https://github.com/microsoft/DeepSpeed
    - https://www.microsoft.com/en-us/research/blog/turing-nlg-a-17-billion-parameter-language-model-by-microsoft/
- huggingface DeepSpeed Integration
    - https://huggingface.co/docs/transformers/main_classes/deepspeed
- pip 安装之后，可以通过 `ds_report` 命令查看环境配置信息；
- bag of tricks
    - $O(n^2)$（$n$ 表示序列长度） => sparse attention；
        - 10x longer seq，up to 6x faster；

## ZeRO: Zero Redundancy Optimizer

## 使用

- https://huggingface.co/docs/transformers/main_classes/deepspeed

### cli

- https://colab.research.google.com/github/stas00/porting/blob/master/transformers/deepspeed/DeepSpeed_on_colab_CLI.ipynb#scrollTo=vSlYvQWLwblN

### inference

- inference-tutorial
    - https://github.com/microsoft/DeepSpeed/blob/master/docs/_tutorials/inference-tutorial.md

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSeq2SeqLM
from transformers.integrations import HfDeepSpeedConfig
import deepspeed
import os
import torch

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
local_rank = int(os.getenv("LOCAL_RANK", "0"))
world_size = int(os.getenv("WORLD_SIZE", "1"))
torch.cuda.set_device(local_rank)
deepspeed.init_distributed()

In [ ]:
model_name = "bigscience/T0_3B"


In [ ]:
config = AutoConfig.from_pretrained(model_name)
model_hidden_size = config.d_model

In [ ]:
train_batch_size = 1 * world_size

In [ ]:
ds_config = {
    "fp16": {
        "enabled": False
    },
    "bf16": {
        "enabled": False
    },
    "zero_optimization": {
        "stage": 3,
        "offload_param": {
            "device": "cpu",
            "pin_memory": True
        },
        "overlap_comm": True,
        "contiguous_gradients": True,
        "reduce_bucket_size": model_hidden_size * model_hidden_size,
        "stage3_prefetch_bucket_size": 0.9 * model_hidden_size * model_hidden_size,
        "stage3_param_persistence_threshold": 10 * model_hidden_size
    },
    "steps_per_print": 2000,
    "train_batch_size": train_batch_size,
    "train_micro_batch_size_per_gpu": 1,
    "wall_clock_breakdown": False
}

In [ ]:
dschf = HfDeepSpeedConfig(ds_config) 